<a href="https://colab.research.google.com/github/inikanipp/Algerian_Forest_Fires_Classification_Prediction/blob/main/model/random_forest_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import library


In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#import dataset

In [85]:
dataset = pd.read_csv('dataset-Algerian.csv')

#Descriptive Statistic

In [86]:
dataset.describe()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI
count,244.000000,244.000000,244.0,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000
mean,15.754098,7.500000,2012.0,32.172131,61.938525,15.504098,0.760656,77.887705,14.673361,49.288484,4.774180,16.664754
std,8.825059,1.112961,0.0,3.633843,14.884200,2.810178,1.999406,14.337571,12.368039,47.619393,4.175318,14.204824
min,1.000000,6.000000,2012.0,22.000000,21.000000,6.000000,0.000000,28.600000,0.700000,6.900000,0.000000,1.100000
25%,8.000000,7.000000,2012.0,30.000000,52.000000,14.000000,0.000000,72.075000,5.800000,13.275000,1.400000,6.000000
50%,16.000000,7.500000,2012.0,32.000000,63.000000,15.000000,0.000000,83.500000,11.300000,33.100000,3.500000,12.250000
75%,23.000000,8.000000,2012.0,35.000000,73.250000,17.000000,0.500000,88.300000,20.750000,68.150000,7.300000,22.525000
max,31.000000,9.000000,2012.0,42.000000,90.000000,29.000000,16.800000,96.000000,65.900000,220.400000,19.000000,68.000000


## jawaban :
dari hasil statistik deskriptif diatas menunjukkan bahwa semua kolom dapat dilihat untuk statistik deskriptifnya, namun terdapat 2 kolom yang tidak dapat dilihat statistik deskriptifnya yaitu kolom FWI dan CLASSES, untuk CLASSES tidak perlu kita cari statistik deskriptifnya karena merupakan kolom target dan berbentuk kategori, namun pada FWI merupakan angka dan perlu kita rubah tipe datanya dari object menjadi float untuk melihat statistik deskriptifnya.

In [87]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          244 non-null    int64  
 1   month        244 non-null    int64  
 2   year         244 non-null    int64  
 3   Temperature  244 non-null    int64  
 4    RH          244 non-null    int64  
 5    Ws          244 non-null    int64  
 6   Rain         244 non-null    float64
 7   FFMC         244 non-null    float64
 8   DMC          244 non-null    float64
 9   DC           244 non-null    float64
 10  ISI          244 non-null    float64
 11  BUI          244 non-null    float64
 12  FWI          244 non-null    object 
 13  Classes      243 non-null    object 
dtypes: float64(6), int64(6), object(2)
memory usage: 26.8+ KB


In [88]:
print(f"nilai unik dari kolom Classes : {dataset['FWI'].unique()}")
print(f"nilai unik dari kolom FWI : {dataset['Classes  '].unique()}")

nilai unik dari kolom Classes : ['0.5' '0.4' '0.1' '0' '2.5' '7.2' '7.1' '0.3' '0.9' '5.6' '7.1 ' '0.2'
 '1.4' '2.2' '2.3' '3.8' '7.5' '8.4' '10.6' '15' '13.9' '3.9' '12.9' '1.7'
 '4.9' '6.8' '3.2' '8' '0.6' '3.4' '0.8' '3.6' '6' '10.9' '4' '8.8' '2.8'
 '2.1' '1.3' '7.3' '15.3' '11.3' '11.9' '10.7' '15.7' '6.1' '2.6' '9.9'
 '11.6' '12.1' '4.2' '10.2' '6.3' '14.6' '16.1' '17.2' '16.8' '18.4'
 '20.4' '22.3' '20.9' '20.3' '13.7' '13.2' '19.9' '30.2' '5.9' '7.7' '9.7'
 '8.3' '0.7' '4.1' '1' '3.1' '1.9' '10' '16.7' '1.2' '5.3' '6.7' '9.5'
 '12' '6.4' '5.2' '3' '9.6' '4.7' 'fire   ' '14.1' '9.1' '13' '17.3' '30'
 '25.4' '16.3' '9' '14.5' '13.5' '19.5' '12.6' '12.7' '21.6' '18.8' '10.5'
 '5.5' '14.8' '24' '26.3' '12.2' '18.1' '24.5' '26.9' '31.1' '30.3' '26.1'
 '16' '19.4' '2.7' '3.7' '10.3' '5.7' '9.8' '19.3' '17.5' '15.4' '15.2'
 '6.5']
nilai unik dari kolom FWI : ['not fire   ' 'fire   ' 'fire' 'fire ' 'not fire' 'not fire '
 'not fire     ' nan 'not fire    ']


## Jawaban :
dari cek informasi tipe data diatas, dapat diketahui bahwa kolom FWI memiliki tipe data object dan perlu kita ubah menjadi float untuk mencari statistic deskriptifnya, dan CLASSES memiliki tipe data object yang seharusnya memiliki tipe data categorical

In [89]:
from logging import exception
errorIndication = []


def tryConvert (row) :
  index = dataset.index.get_loc(row.name)
  try :
    dataset.iloc[index,-2] = float(dataset.iloc[index, -2])
    # print("ini adalah", row.name)
    # print(dataset.iloc[index,-2])


  except Exception:
        errorIndication.append(index)

dataset.apply( tryConvert ,axis=1)

print(errorIndication)

for i in errorIndication :
  dataset.drop(i, inplace = True)
  print(i)


[165]
165


# cek apakah baris seudah berhasil dihapus

In [95]:
dataset.shape

(243, 14)

## keterangan :
dari hasil diatas dapat kita cek bahwa terdapat error pada baris ke 165 ketika mengubah ke tipe data float, sehingga kita bisa menghapus baris tersebut dan mengubah tipe data kolom menjadi sebuah float

#ubah kolom ke float dan mengeceknya kembali

In [93]:
dataset['FWI'] = dataset['FWI'].astype('float')

In [94]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243 entries, 0 to 243
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          243 non-null    int64  
 1   month        243 non-null    int64  
 2   year         243 non-null    int64  
 3   Temperature  243 non-null    int64  
 4    RH          243 non-null    int64  
 5    Ws          243 non-null    int64  
 6   Rain         243 non-null    float64
 7   FFMC         243 non-null    float64
 8   DMC          243 non-null    float64
 9   DC           243 non-null    float64
 10  ISI          243 non-null    float64
 11  BUI          243 non-null    float64
 12  FWI          243 non-null    float64
 13  Classes      243 non-null    object 
dtypes: float64(7), int64(6), object(1)
memory usage: 28.5+ KB


# cek nilai unik kolom classses

In [96]:
dataset['Classes  '].unique()

array(['not fire   ', 'fire   ', 'fire', 'fire ', 'not fire', 'not fire ',
       'not fire     ', 'not fire    '], dtype=object)

## keterangan :
dari output diatas dapat kita amati bahwa data tersebut sebenarnya hanya memiliki 2 kategori namun terdapat perbedaan penulisan sehingga perlu untuk kita cleaning

#Cleaning data in classes column

In [98]:
dataset['Classes  '] = dataset['Classes  '].str.strip()
dataset['Classes  '] = dataset['Classes  '].astype('category')

In [99]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243 entries, 0 to 243
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   day          243 non-null    int64   
 1   month        243 non-null    int64   
 2   year         243 non-null    int64   
 3   Temperature  243 non-null    int64   
 4    RH          243 non-null    int64   
 5    Ws          243 non-null    int64   
 6   Rain         243 non-null    float64 
 7   FFMC         243 non-null    float64 
 8   DMC          243 non-null    float64 
 9   DC           243 non-null    float64 
 10  ISI          243 non-null    float64 
 11  BUI          243 non-null    float64 
 12  FWI          243 non-null    float64 
 13  Classes      243 non-null    category
dtypes: category(1), float64(7), int64(6)
memory usage: 26.9 KB


In [101]:
dataset['Classes  '].unique()

['not fire', 'fire']
Categories (2, object): ['fire', 'not fire']